#### Lesson 5: Coding and Math Problem Solving

The python version of this lesson is Coding and Financial Analysis. Because of lacking financial and plotting library in dotnet, it is very challenging for GPT to analyze financial data using C# as programming language. So I change the topic to Coding and Math Problem Solving.

#### Setup

In [1]:
#r "nuget:AutoGen,0.2.0"
#r "nuget:AutoGen.DotnetInteractive,0.2.0"
#load "util.csx"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using AutoGen.DotnetInteractive;
using AutoGen.DotnetInteractive.Extension;
using OpenAI;
using OpenAI.Chat;
using System.Threading;
using System.IO;
using System.Reactive.Linq;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Events;

var chatClient = ChatClientProvider.Create("gpt-4o-mini");

Installed Packages AutoGen, 0.2.0 AutoGen.DotnetInteractive, 0.2.0

Agent with code executor configuration. Because we are in dotnet interactive notebook, we can simply using the dotnet interactive kernel to execute the code.

The code below retreive the csharp code between ```csharp and  ``` and send it to dotnet interactive kernel to execute.

In [2]:
// Agent with code executor configuration
var codeExecutorAgent = new DefaultReplyAgent(
    name: "code_executor_agent",
    defaultReply: "no code to execute")
    .RegisterMiddleware(async (msgs, option, next, ct) =>
    {
        // check if the last message contain csharp code blcok
        var lastMsg = msgs.LastOrDefault();
        if (lastMsg?.ExtractCodeBlock("```csharp", "```") is string csharpCode)
        {
            var events = new List<DisplayEvent>();
            Kernel.Current.RootKernel.KernelEvents.OfType<DisplayEvent>().Subscribe(e => events.Add(e));
            var kernelCommandResult = await Kernel.Current.RootKernel.SubmitCodeAsync(csharpCode);
            var displayValues = events.Where(x => x is StandardErrorValueProduced || x is StandardOutputValueProduced || x is ReturnValueProduced)
                .SelectMany(x => (x as DisplayEvent)!.FormattedValues);
            
            if (displayValues is null || displayValues.Count() == 0)
            {
                return new TextMessage(Role.Assistant, "No output", from: next.Name);
            }
            
            var codeResult = string.Join("\n", displayValues.Select(x => x.Value));
            
            return new TextMessage(Role.Assistant, codeResult ?? string.Empty, from: next.Name);
        }

        return await next.GenerateReplyAsync(msgs, option, ct);
    })
    .RegisterPrintMessage();

Agent with dotnet coding writing capability

In [3]:
var coderAgent = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "code_writer_agent",
    systemMessage: """
    You act as dotnet coder, you write dotnet code to resolve task. Once you finish writing code, ask runner to run the code for you.

    Here're some rules to follow on writing dotnet code:
    - put code between ```csharp and ```
    - When creating http client, use `var httpClient = new HttpClient()`. Don't use `using var httpClient = new HttpClient()` because it will cause error when running the code.
    - Try to use `var` instead of explicit type.
    - Try avoid using external library, use .NET Core library instead.
    - Use top level statement to write code.
    - Always print out the result to console. Don't write code that doesn't print out anything.
    
    If you need to install nuget packages, put nuget packages in the following format:
    ```nuget
    nuget_package_name
    ```
    
    If your code is incorrect, Fix the error and send the code again.
    Once the task is resolved, say 'task completed' to finish the task.
    """,
    temperature: 0.4f)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

The task!

In [4]:
var task = """
    calculate the 39th fibonacci number and save the result to result.txt
    """;

Start the conversation to resolve the task.

In [5]:
var chatHistory = new List<IMessage>()
{
    new TextMessage(Role.Assistant, task, from: codeExecutorAgent.Name),
};

await foreach(var reply in coderAgent.SendAsync(receiver: codeExecutorAgent, chatHistory: chatHistory, maxRound: 10))
{

    if (reply.GetContent()?.ToLower().Contains("task completed") == true)
    {
        break;
    }
    else
    {
        chatHistory.Add(reply);
    }
}

from: code_writer_agent
```csharp
using System;
using System.IO;

var fibonacci = new long[40];
fibonacci[0] = 0;
fibonacci[1] = 1;

for (var i = 2; i <= 39; i++)
{
    fibonacci[i] = fibonacci[i - 1] + fibonacci[i - 2];
}

var result = fibonacci[39];
File.WriteAllText("result.txt", result.ToString());
Console.WriteLine($"The 39th Fibonacci number is: {result}");
```

Please run the code.

The 39th Fibonacci number is: 63245986
TextMessage from code_executor_agent
--------------------
The 39th Fibonacci number is: 63245986

--------------------

from: code_writer_agent
Task completed.

